# ALBERT Q&A Training

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from scripts import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # Oliviera's script
from scripts.tf2_0_baseline_w_bert_translated_to_tf2_0 import AnswerType
from scripts import albert_tokenization as tokenization

import absl
import collections
import json
import sys
import threading
import time
import tqdm
import zipfile

tf2baseline.FLAGS.include_unknowns = -1

TF 2.0 Baseline Loaded


## Define Flags

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(absl.flags.FLAGS)

flags = absl.flags

flags.DEFINE_string("vocab_file", "models/albert_xxl/vocab/modified-30k-clean.model",
                    "The vocabulary file that the BERT/ALBERT model was trained on.")

flags.DEFINE_string(
    "output_dir", "output/",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("train_precomputed_file", "albert_train.tf_record",
                    "Precomputed tf records for training.")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length", 256,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "doc_stride", 128,
    "When splitting up a long document into chunks, how much stride to "
    "take between chunks.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum number of tokens for the question. Questions longer than "
    "this will be truncated to this length.")

flags.DEFINE_bool("do_train", True, "Whether to run training.")

flags.DEFINE_bool("do_predict", False, "Whether to run eval on the dev set.")

flags.DEFINE_integer(
    "max_answer_length", 30,
    "The maximum length of an answer that can be generated. This is needed "
    "because the start and end predictions are not conditioned on one another.")

flags.DEFINE_float(
    "include_unknowns", -1,
    "If positive, probability of including answers of type `UNKNOWN`.")

flags.DEFINE_boolean(
    "skip_nested_contexts", True,
    "Completely ignore context that are not top level nodes in the page.")

flags.DEFINE_integer("max_contexts", 48,
                     "Maximum number of contexts to output for an example.")

flags.DEFINE_integer(
    "max_position", 50,
    "Maximum context position for which to generate special tokens.")

## Custom flags

flags.DEFINE_integer(
    "n_examples", -1,
    "Number of examples to read from files. Only applicable during testing")

flags.DEFINE_string(
    "train_file", "data/simplified-nq-train.jsonl.zip",
    "NQ json for training. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")

## Special flags - do not change

flags.DEFINE_string(
    "predict_file", "/home/ejmejm/MLProjects/nqa_kaggle/data/simplified-nq-test.jsonl",
    "NQ json for predictions. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")
flags.DEFINE_boolean("logtostderr", True, "Logs to stderr")
flags.DEFINE_boolean("undefok", True, "it's okay to be undefined")
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('HistoryManager.hist_file', '', 'kernel')

FLAGS = flags.FLAGS
FLAGS(sys.argv) # Parse the flags

['/home/ejmejm/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py']

## Generate Formatted Training Data (TFRecord, Only Once)

In [3]:
def blocks(f, size=65536):
    while True:
        b = f.read(size)
        if not b:
            break
        yield b
    
with zipfile.ZipFile(FLAGS.train_file) as zip_file:
    with zip_file.open('simplified-nq-train.jsonl', 'r') as f:
        n_train_examples = sum([bl.decode('UTF-8').count('\n') for bl in blocks(f)])

print('# Training Examples:', n_train_examples)

# Training Examples: 307373


In [4]:
def data_generator(chunk_size=1000):
    curr_pos = 0
    last_line = False
    with zipfile.ZipFile(FLAGS.train_file) as zip_file:
        with zip_file.open('simplified-nq-train.jsonl', 'r') as f:
            while not last_line:
                examples = []
                for i in range(curr_pos, curr_pos+chunk_size):
                    line = f.readline().decode('UTF-8')
                    if line is None:
                        last_line = True
                        break
                    examples.append(tf2baseline.create_example_from_jsonl(line, lowercase=True))
                    examples[-1] = tf2baseline.read_nq_entry(examples[-1], FLAGS.do_train)[0]
                curr_pos = i + 1
                yield examples

In [5]:
def example_to_verifier_feature(example, tokenizer, correct):
    question_text = example.questions[0]
    answer_text = example.answer.text
    if answer_text == '[':
        answer_text = None
        
    question_tokens = tokenizer.tokenize(question_text)
    answer_tokens = tokenizer.tokenize(answer_text)
    
    if len(question_tokens) > FLAGS.max_query_length:
        question_tokens = question_tokens[-FLAGS.max_query_length:]
        
    max_answer_length = FLAGS.max_seq_length - (FLAGS.max_query_length + 4)
    if len(answer_tokens) > max_answer_length:
        answer_tokens = answer_tokens[:max_answer_length]
        
    n_q_tokens = len(question_tokens) + 3
    n_a_tokens = len(answer_tokens) + 1
    n_pad_tokens = FLAGS.max_seq_length - (n_q_tokens + n_a_tokens)

    input_tokens = ['[CLS]', '[Q]'] + \
                 question_tokens + \
                 ['[SEP]'] + \
                 answer_tokens + \
                 ['[SEP]'] + \
                 ['[PAD]'] * n_pad_tokens
    
    
    
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    segment_ids = np.concatenate([np.zeros(n_q_tokens, dtype=np.int32),
                                  np.ones(n_a_tokens, dtype=np.int32),
                                  np.zeros(n_pad_tokens, dtype=np.int32)])
    input_mask = np.concatenate([np.ones(FLAGS.max_seq_length - n_pad_tokens, dtype=np.int32),
                                 np.zeros(n_pad_tokens, dtype=np.int32)])
    
    assert len(input_ids) == FLAGS.max_seq_length
    assert len(segment_ids) == FLAGS.max_seq_length
    assert len(input_mask) == FLAGS.max_seq_length
    
    if correct:
        valid = [1, 0]
    else:
        valid = [0, 1]
    
    return {'input_ids': input_ids,
            'segment_ids': segment_ids,
            'input_mask': input_mask,
            'valid': valid}

In [6]:
def encode_example(features):
    example = tf.train.Example(features=tf.train.Features(feature=features))
    return example.SerializeToString()

def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list=tf.train.Int64List(value=list(values)))
    return feature

def dict_to_feature(d):
    od = collections.OrderedDict()
    for k, v in d.items():
        od[k] = create_int_feature(v)
    return encode_example(od)

In [12]:
chunk_size = 400
example_gen = data_generator(chunk_size=chunk_size)

writer = tf.io.TFRecordWriter(os.path.join(FLAGS.output_dir, 'verifier_train.tf_record'))
tokenizer = tokenization.FullTokenizer(
    None,
    spm_model_file=FLAGS.vocab_file)

try:
    n_batches = int(np.ceil(n_train_examples / chunk_size))
    for _ in tqdm.tqdm(range(n_batches)):
        examples = next(example_gen)
        examples = [example for example in examples if example.answer.type != AnswerType.UNKNOWN]
        for i in range(0, len(examples), 2):
            try:
                if i + 1 >= len(examples):
                    feature_data = example_to_verifier_feature(examples[i], tokenizer, True)
                    feature = dict_to_feature(feature_data)
                    writer.write(feature)
                    break

                correct = True
                for switch in range(2):
                    if switch == 1:
                        tmp_answer = examples[i].answer
                        examples[i].answer = examples[i+1].answer
                        examples[i+1].answer = tmp_answer
                        correct = False

                    feature_data_1 = example_to_verifier_feature(examples[i], tokenizer, correct)
                    feature_data_2 = example_to_verifier_feature(examples[i+1], tokenizer, correct)

                    feature_1 = dict_to_feature(feature_data_1)
                    feature_2 = dict_to_feature(feature_data_2)

                    writer.write(feature_1)
                    writer.write(feature_2)
            except:
                print('Ran into error with examples {} and {}, skipping for now.'.format(
                      examples[i].example_id, examples[i+1].example_id))
                continue
finally:
    writer.close()




  0%|          | 0/769 [00:00<?, ?it/s]


  0%|          | 1/769 [00:07<1:40:11,  7.83s/it]


  0%|          | 2/769 [00:15<1:41:04,  7.91s/it]


  0%|          | 3/769 [00:24<1:41:42,  7.97s/it]


  1%|          | 4/769 [00:32<1:42:27,  8.04s/it]


  1%|          | 5/769 [00:40<1:42:20,  8.04s/it]


  1%|          | 6/769 [00:47<1:39:26,  7.82s/it]


  1%|          | 7/769 [00:55<1:39:08,  7.81s/it]


  1%|          | 8/769 [01:02<1:37:08,  7.66s/it]


  1%|          | 9/769 [01:10<1:37:09,  7.67s/it]


  1%|▏         | 10/769 [01:17<1:36:39,  7.64s/it]


  1%|▏         | 11/769 [01:25<1:35:16,  7.54s/it]


  2%|▏         | 12/769 [01:32<1:35:04,  7.54s/it]


  2%|▏         | 13/769 [01:40<1:35:51,  7.61s/it]


  2%|▏         | 14/769 [01:47<1:33:58,  7.47s/it]


  2%|▏         | 15/769 [01:55<1:35:42,  7.62s/it]


  2%|▏         | 16/769 [02:03<1:37:17,  7.75s/it]


  2%|▏         | 17/769 [02:11<1:36:25,  7.69s/it]


  2%|▏         | 18/769 [02:19<1:37:14,  7.77s/it]


  2%|▏    

Ran into error with examples -7931234188600504200 and 4102002209886373355, skipping for now.





 13%|█▎        | 99/769 [12:49<1:25:47,  7.68s/it]


 13%|█▎        | 100/769 [12:57<1:26:20,  7.74s/it]


 13%|█▎        | 101/769 [13:06<1:29:34,  8.05s/it]


 13%|█▎        | 102/769 [13:14<1:29:18,  8.03s/it]


 13%|█▎        | 103/769 [13:21<1:28:06,  7.94s/it]


 14%|█▎        | 104/769 [13:29<1:28:04,  7.95s/it]


 14%|█▎        | 105/769 [13:37<1:26:20,  7.80s/it]


 14%|█▍        | 106/769 [13:44<1:24:04,  7.61s/it]


 14%|█▍        | 107/769 [13:51<1:23:14,  7.54s/it]


 14%|█▍        | 108/769 [13:58<1:21:27,  7.39s/it]


 14%|█▍        | 109/769 [14:06<1:21:58,  7.45s/it]


 14%|█▍        | 110/769 [14:13<1:21:55,  7.46s/it]


 14%|█▍        | 111/769 [14:21<1:23:04,  7.58s/it]

Ran into error with examples -3697770822124923277 and 6121161464638517591, skipping for now.





 15%|█▍        | 112/769 [14:30<1:25:11,  7.78s/it]


 15%|█▍        | 113/769 [14:38<1:27:44,  8.03s/it]


 15%|█▍        | 114/769 [14:48<1:34:12,  8.63s/it]


 15%|█▍        | 115/769 [14:56<1:32:13,  8.46s/it]


 15%|█▌        | 116/769 [15:04<1:30:21,  8.30s/it]


 15%|█▌        | 117/769 [15:11<1:25:45,  7.89s/it]


 15%|█▌        | 118/769 [15:18<1:22:41,  7.62s/it]


 15%|█▌        | 119/769 [15:26<1:22:48,  7.64s/it]


 16%|█▌        | 120/769 [15:33<1:22:18,  7.61s/it]


 16%|█▌        | 121/769 [15:40<1:19:48,  7.39s/it]


 16%|█▌        | 122/769 [15:48<1:21:20,  7.54s/it]


 16%|█▌        | 123/769 [15:56<1:23:06,  7.72s/it]


 16%|█▌        | 124/769 [16:04<1:24:15,  7.84s/it]


 16%|█▋        | 125/769 [16:13<1:27:19,  8.14s/it]


 16%|█▋        | 126/769 [16:22<1:28:45,  8.28s/it]


 17%|█▋        | 127/769 [16:30<1:27:34,  8.19s/it]


 17%|█▋        | 128/769 [16:38<1:28:24,  8.28s/it]


 17%|█▋        | 129/769 [16:46<1:26:15,  8.09s/it]


 17%|█▋        | 130/769 

...

Ran into error with examples 5326965434046307540 and 1720859761886088300, skipping for now.





 95%|█████████▍| 728/769 [1:33:05<05:17,  7.74s/it]


 95%|█████████▍| 729/769 [1:33:12<05:07,  7.68s/it]


 95%|█████████▍| 730/769 [1:33:20<05:02,  7.76s/it]


 95%|█████████▌| 731/769 [1:33:27<04:49,  7.63s/it]


 95%|█████████▌| 732/769 [1:33:35<04:39,  7.56s/it]


 95%|█████████▌| 733/769 [1:33:43<04:35,  7.66s/it]


 95%|█████████▌| 734/769 [1:33:50<04:25,  7.58s/it]


 96%|█████████▌| 735/769 [1:33:58<04:17,  7.57s/it]

Ran into error with examples 272062084415668460 and -5564280557450371074, skipping for now.





 96%|█████████▌| 736/769 [1:34:07<04:25,  8.04s/it]


 96%|█████████▌| 737/769 [1:34:15<04:19,  8.12s/it]


 96%|█████████▌| 738/769 [1:34:22<04:03,  7.87s/it]


 96%|█████████▌| 739/769 [1:34:30<03:49,  7.66s/it]


 96%|█████████▌| 740/769 [1:34:37<03:42,  7.67s/it]


 96%|█████████▋| 741/769 [1:34:45<03:33,  7.62s/it]


 96%|█████████▋| 742/769 [1:34:53<03:28,  7.71s/it]


 97%|█████████▋| 743/769 [1:35:01<03:21,  7.75s/it]


 97%|█████████▋| 744/769 [1:35:08<03:10,  7.62s/it]


 97%|█████████▋| 745/769 [1:35:15<03:02,  7.60s/it]


 97%|█████████▋| 746/769 [1:35:23<02:56,  7.69s/it]


 97%|█████████▋| 747/769 [1:35:31<02:49,  7.68s/it]


 97%|█████████▋| 748/769 [1:35:39<02:44,  7.82s/it]


 97%|█████████▋| 749/769 [1:35:46<02:32,  7.64s/it]


 98%|█████████▊| 750/769 [1:35:55<02:30,  7.92s/it]


 98%|█████████▊| 751/769 [1:36:03<02:20,  7.82s/it]


 98%|█████████▊| 752/769 [1:36:10<02:10,  7.70s/it]


 98%|█████████▊| 753/769 [1:36:18<02:03,  7.73s/it]


 98%|█████████▊| 754/769 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)